# 11. 3개월 폐업 예측 모델

## 목적
- reference.ipynb 섹션 4-4 방법론을 적용하여 3개월 내 폐업 예측 모델 구축
- 3가지 모델(LogisticRegression, HistGradientBoosting, RandomForest) 비교
- ROC-AUC, PR-AUC, F1 Score로 성능 평가

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

from sklearn.metrics import (
    roc_auc_score, 
    average_precision_score, 
    f1_score,
    classification_report
)

from sklearn.pipeline import Pipeline
from sklearn.utils import resample

print("라이브러리 로드 완료")

라이브러리 로드 완료


## 1. 데이터 로드 및 타깃 변수 생성

In [2]:
# 데이터 경로
data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
result_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"

# 전체 데이터 로드
df = pd.read_csv(data_path)
print(f"전체 데이터 형태: {df.shape}")

# 클러스터링 결과 로드
cluster_result = pd.read_csv(f"{result_dir}/클러스터링_결과_완전판.csv")
print(f"클러스터링 결과 형태: {cluster_result.shape}")

전체 데이터 형태: (86263, 188)
클러스터링 결과 형태: (4325, 4)


In [3]:
# 타깃 변수 생성: y_close_3m
# 상태가 'D-3m'이면 1, 아니면 0

df_model = cluster_result.merge(df, on='가맹점구분번호', how='left')

# 타깃 변수 (D-3m만 1, 나머지 0)
df_model['y_close_3m'] = (df_model['상태'] == 'D-3m').astype(int)

print("\n[타깃 변수 분포]")
print(df_model['y_close_3m'].value_counts())
print(f"\n폐업 비율: {df_model['y_close_3m'].mean()*100:.2f}%")


[타깃 변수 분포]
y_close_3m
0    88946
1      457
Name: count, dtype: int64

폐업 비율: 0.51%


## 2. 피처 선택 및 전처리

In [4]:
# 제외할 컬럼 (식별자, 타깃, 날짜 등)
exclude_cols = {
    '가맹점구분번호', '기준년월', '기준연월', '기준분기', '개설일', '폐업일',
    '상태', 'y_close_3m', 'cluster',
    '좌표정보(X)', '좌표정보(Y)', '상권_코드', '상권_코드_명'
}

# 사용 가능한 피처 선택
all_cols = set(df_model.columns)
feature_cols = list(all_cols - exclude_cols)

# 수치형 피처만 선택
numeric_features = []
for col in feature_cols:
    if col in df_model.columns:
        if pd.api.types.is_numeric_dtype(df_model[col]):
            numeric_features.append(col)

print(f"\n[선택된 피처 개수]: {len(numeric_features)}")
print(f"\n[상위 10개 피처]:")
for i, col in enumerate(numeric_features[:10], 1):
    print(f"{i}. {col}")


[선택된 피처 개수]: 168

[상위 10개 피처]:
1. CPI_품목_월세
2. 동일 업종 매출건수 비율
3. CPI_목적_음식및숙박_숙박서비스
4. CPI_목적_기타상품및서비스_미용용품및미용서비스
5. CPI_품목상세_서비스_공공서비스
6. CPI_품목_국산쇠고기
7. CPI_품목_치과진료비
8. 식료품_지출_총금액
9. CPI_품목_휘발유
10. 생활물가지수


In [5]:
# 결측치 처리
X = df_model[numeric_features].copy()
y = df_model['y_close_3m'].copy()

# 결측치를 중앙값으로 대치
for col in X.columns:
    if X[col].isna().sum() > 0:
        X[col] = X[col].fillna(X[col].median())

# 무한대 값 처리
X = X.replace([np.inf, -np.inf], np.nan)
for col in X.columns:
    if X[col].isna().sum() > 0:
        X[col] = X[col].fillna(X[col].median())

print(f"\n최종 데이터 형태: X={X.shape}, y={y.shape}")
print(f"결측치 개수: {X.isna().sum().sum()}")


최종 데이터 형태: X=(89403, 168), y=(89403,)
결측치 개수: 0


## 3. Train/Test Split

In [6]:
# 80:20 split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain set: {X_train.shape}, 폐업률={y_train.mean()*100:.2f}%")
print(f"Test set: {X_test.shape}, 폐업률={y_test.mean()*100:.2f}%")


Train set: (71522, 168), 폐업률=0.51%
Test set: (17881, 168), 폐업률=0.51%


## 4. 모델 구축 (reference.ipynb 방법론)

### 3가지 모델:
1. **LogisticRegression**: 베이스라인
2. **HistGradientBoostingClassifier**: 권장 모델
3. **RandomForestClassifier**: 랜덤 오버샘플링

In [7]:
# 불균형 비율 계산
pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"클래스 불균형 비율 (0:1) = {pos_weight:.1f}:1")

# 전처리 파이프라인 (스케일링)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

preproc = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features)
    ],
    remainder='drop'
)

# 수동 오버샘플링 함수
def manual_oversample(X, y, random_state=42):
    """소수 클래스를 다수 클래스와 같은 수로 오버샘플링"""
    # 클래스별로 분리
    X_maj = X[y == 0]
    X_min = X[y == 1]
    y_maj = y[y == 0]
    y_min = y[y == 1]
    
    # 소수 클래스 오버샘플링
    X_min_upsampled = resample(X_min, 
                                replace=True,
                                n_samples=len(X_maj),
                                random_state=random_state)
    y_min_upsampled = np.ones(len(X_maj), dtype=int)
    
    # 합치기
    X_resampled = pd.concat([X_maj, pd.DataFrame(X_min_upsampled, columns=X_min.columns)])
    y_resampled = np.concatenate([y_maj, y_min_upsampled])
    
    return X_resampled, y_resampled

print("전처리 파이프라인 및 오버샘플링 함수 정의 완료")

클래스 불균형 비율 (0:1) = 194.4:1
전처리 파이프라인 및 오버샘플링 함수 정의 완료


In [8]:
# 1) Logistic Regression (베이스라인)
logit = Pipeline(steps=[
    ('pre', preproc),
    ('clf', LogisticRegression(
        max_iter=1000,
        random_state=42,
        class_weight='balanced'
    ))
])

# 2) HistGradientBoosting (권장 모델)
hgb = Pipeline(steps=[
    ('pre', preproc),
    ('clf', HistGradientBoostingClassifier(
        learning_rate=0.08,
        max_depth=None,
        max_leaf_nodes=31,
        min_samples_leaf=50,
        random_state=42
    ))
])

# 3) RandomForest + 수동 오버샘플링
# 오버샘플링은 학습 시 직접 적용
rf_clf = Pipeline(steps=[
    ('pre', preproc),
    ('clf', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        random_state=42,
        n_jobs=-1
    ))
])

models = {
    'logit': logit,
    'hgb': hgb,
    'rf_bal': rf_clf  # 오버샘플링은 학습 시 적용
}

print("\n[모델 정의 완료]")
for name in models:
    print(f"- {name}")


[모델 정의 완료]
- logit
- hgb
- rf_bal


## 5. 모델 학습 및 평가

In [9]:
# 모델 학습 및 평가
results = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"[{name}] 학습 시작...")
    print(f"{'='*60}")
    
    # rf_bal의 경우 오버샘플링 적용
    if name == 'rf_bal':
        print("  오버샘플링 적용 중...")
        X_train_balanced, y_train_balanced = manual_oversample(X_train, y_train, random_state=42)
        print(f"  오버샘플링 후: {X_train_balanced.shape}, 폐업률={y_train_balanced.mean()*100:.2f}%")
        model.fit(X_train_balanced, y_train_balanced)
    else:
        model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)
    
    # 확률 예측
    if hasattr(model[-1], 'predict_proba'):
        y_proba = model.predict_proba(X_test)[:, 1]
    else:
        y_proba = model.decision_function(X_test)
    
    # 평가 지표 계산
    roc = roc_auc_score(y_test, y_proba)
    pr = average_precision_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)
    
    results[name] = {
        'ROC-AUC': roc,
        'PR-AUC': pr,
        'F1@0.5': f1
    }
    
    print(f"\n[{name}] ROC-AUC={roc:.3f}  PR-AUC={pr:.3f}  F1@0.5={f1:.3f}")
    print(classification_report(y_test, y_pred, digits=3))

print("\n" + "="*60)
print("[모든 모델 학습 완료]")
print("="*60)


[logit] 학습 시작...

[logit] ROC-AUC=1.000  PR-AUC=1.000  F1@0.5=1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000     17790
           1      1.000     1.000     1.000        91

    accuracy                          1.000     17881
   macro avg      1.000     1.000     1.000     17881
weighted avg      1.000     1.000     1.000     17881


[hgb] 학습 시작...

[hgb] ROC-AUC=1.000  PR-AUC=1.000  F1@0.5=1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000     17790
           1      1.000     1.000     1.000        91

    accuracy                          1.000     17881
   macro avg      1.000     1.000     1.000     17881
weighted avg      1.000     1.000     1.000     17881


[rf_bal] 학습 시작...
  오버샘플링 적용 중...
  오버샘플링 후: (142312, 168), 폐업률=50.00%

[rf_bal] ROC-AUC=1.000  PR-AUC=0.993  F1@0.5=0.829
              precision    recall  f1-score   support

           0      1.000     0.998

## 6. 결과 요약

In [10]:
# 결과 DataFrame 생성
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('ROC-AUC', ascending=False)

print("\n" + "="*60)
print("[최종 성능 비교]")
print("="*60)
print(results_df.round(3))

# Best model 선택
best_name = results_df.index[0]
best_score = results_df.loc[best_name, 'ROC-AUC']

print(f"\n🏆 Best Model (ROC-AUC 기준): {best_name} (ROC-AUC={best_score:.3f})")


[최종 성능 비교]
        ROC-AUC  PR-AUC  F1@0.5
logit       1.0   1.000   1.000
hgb         1.0   1.000   1.000
rf_bal      1.0   0.993   0.829

🏆 Best Model (ROC-AUC 기준): logit (ROC-AUC=1.000)


## 7. 결과 저장

In [11]:
# 결과 저장
results_df.to_csv(f"{result_dir}/11_모델성능_비교결과.csv", encoding='utf-8-sig')
print(f"\n✅ 결과 저장 완료: {result_dir}/11_모델성능_비교결과.csv")


✅ 결과 저장 완료: /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석/11_모델성능_비교결과.csv


## 8. 해석

**실제 실행 결과를 바탕으로**:
- 3가지 모델의 성능을 직접 비교
- ROC-AUC, PR-AUC, F1 Score로 평가
- Best Model을 선정하여 보고서에 반영